In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
import numpy as np
import pandas as pd
import xlsxwriter
from time import sleep

In [25]:
# получение списка всех регионов
url = 'https://education.yandex.ru/universities/'
text = requests.get(url).text

p = re.compile(r'\"id\":(\d{3}),\"title\":\"([\w ()-—]+)\",\"geoId\"')
pairs = p.findall(text)

regions = {}
regions_json_form = {}
for key, value in pairs:
    key = int(key)
    #print('tree.add(' + "'" + value + "')")
    regions.update({key: value})
    regions_json_form.update({key: {'name': value, 'universities': []}})

regions
#print(regions_json_form)


{438: 'Алтайский край',
 439: 'Амурская область',
 440: 'Архангельская область',
 441: 'Астраханская область',
 442: 'Белгородская область',
 443: 'Брянская область',
 444: 'Владимирская область',
 445: 'Волгоградская область',
 446: 'Вологодская область',
 447: 'Воронежская область',
 448: 'Еврейская автономная область',
 449: 'Забайкальский край',
 450: 'Ивановская область',
 451: 'Иркутская область',
 452: 'Кабардино-Балкарская Республика',
 453: 'Калининградская область',
 454: 'Калужская область',
 455: 'Камчатский край',
 456: 'Карачаево-Черкесская Республика',
 457: 'Кемеровская область',
 458: 'Кировская область',
 459: 'Костромская область',
 460: 'Краснодарский край',
 461: 'Красноярский край',
 462: 'Курганская область',
 463: 'Курская область',
 464: 'Ленинградская область',
 465: 'Липецкая область',
 466: 'Магаданская область',
 467: 'Москва и Московская область',
 468: 'Мурманская область',
 469: 'Ненецкий автономный округ',
 470: 'Нижегородская область',
 471: 'Новгородс

In [19]:
# получение списка всех специальностей
url = 'https://education.yandex.ru/universities/'
print(requests.get(url).ok)
text = requests.get(url).text

p = re.compile(r'\"id\":(\d{4}),\"code\":\"\d{2}.\d{2}.\d{2}\",\"title\":\"([\w .,()-:]+)\",\"educationTypes\"')
pairs = p.findall(text)

specialties = {}
specialties_id = []
for key, value in pairs:
    key = int(key)
    specialties.update({key: value})
    specialties_id.append(key)

specialties

url = 'http://vuzoteka.ru/%D0%B2%D1%83%D0%B7%D1%8B/%D1%81%D0%BF%D0%B5%D1%86%D0%B8%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8'

soup = BeautifulSoup(requests.get(url).text, 'html.parser')

specialties_rating = {}

specialties_raw = soup.findAll('div', attrs={'style': ['margin:0px 0 15px 0px; border:1px solid #eee; border-radius:5px;']})
for specialty in specialties_raw:
    name = specialty.find('a').text
    place = re.match(r'рейтинг\s\(баллы\)\s(\d+)\s\(\d+\)', specialty.find('div', attrs={'class': ['label-part last']}).text).group(1)
    specialties_rating.update({name: int(place)})

for key, value in specialties.items():
    print(value, specialties_rating.get(value, None))


True
Математика 17
Прикладная математика и информатика 2
Механика и математическое моделирование 85
Прикладная математика 24
Фундаментальные математика и механика 174
Математика и компьютерные науки 40
Фундаментальная информатика и информационные технологии 26
Математическое обеспечение и администрирование информационных систем 73
Прикладные математика и физика 23
Физика 25
Радиофизика 83
Астрономия 115
Химия 46
Химия, физика и механика материалов 155
Фундаментальная и прикладная химия 37
Геология 51
География 39
Картография и геоинформатика 154
Гидрометеорология 70
Прикладная гидрометеорология 214
Экология и природопользование 22
Метеорология специального назначения None
Военная картография None
Биология 26
Почвоведение 82
Биоинженерия и биоинформатика 178
Архитектура 106
Реконструкция и реставрация архитектурного наследия 217
Дизайн архитектурной среды 140
Градостроительство 216
Строительство 79
Строительство уникальных зданий и сооружений 86
Строительство железных дорог, мостов и тр

In [ ]:

# набор функций для парсинга страницы
def get_level(td_list):
    return td_list[0].attrs['title']

def get_specialty(td_list):
    specialty = td_list[1].text
    pos = specialty.find('«') + 1
    if pos:
        specialty = specialty[pos:specialty.find('»')]
    return specialty

def get_department(td_list):
    return td_list[1].text

def get_subjects(td_list):
    subjects = []
    for subject in td_list[2].findAll('div', attrs={'class': ['year-stats-subjects-popup__subject']}):
        if subject.attrs['class'] == ['year-stats-subjects-popup__subject']:
            subjects.append(subject.text)
    return subjects

def get_budget_plan(td_list):
    budget_plan = None
    if td_list[3].find('div') != None:
        budget_plan = td_list[3].find('div', attrs={'class': ['year-stats-cell-popup__title']}).text
    return budget_plan

def get_budget(td_list):
    budget = None
    if td_list[3].find('div') != None:
        budget = {}
        budget_plan = td_list[3].find('div', attrs={'class': ['year-stats-cell-popup__title']}).text
        for row in td_list[3].findAll('div', attrs={'class': ['year-stats-places-popup__row']}):
            budget.update({row.find('span').text: row.find('div', attrs={'class': ['year-stats-places-popup__row-value']}).text})
    return budget

def get_budget_passing_score(td_list):
    budget_passing_score = None
    text = td_list[4].find('div', attrs={'class': ['year-stats-points-popup__points']}).text
    if text != '':
        budget_passing_score = text    
    return budget_passing_score

def get_paid_plan(td_list):
    paid_plan = None
    if td_list[5].find('div') != None:
        paid_plan = td_list[5].find('div', attrs={'class': ['year-stats-cell-popup__title']}).text
    return paid_plan

def get_paid(td_list):
    paid = None
    if td_list[5].find('div') != None:
        paid = {}
        paid_plan = td_list[5].find('div', attrs={'class': ['year-stats-cell-popup__title']}).text
        for row in td_list[5].findAll('div', attrs={'class': ['year-stats-places-popup__row']}):
            paid.update({row.find('span').text: row.find('div', attrs={'class': ['year-stats-places-popup__row-value']}).text})
    return paid

def get_paid_passing_score(td_list):
    paid_passing_score = None
    text = td_list[6].find('div', attrs={'class': ['year-stats-table-row__right-aligned']}).text
    if text != '':
        paid_passing_score = text
    return paid_passing_score

def get_cost(td_list):
    return td_list[7].find('div', attrs={'class': ['year-stats-table-row__right-aligned']}).text.replace(' ', '')

In [7]:
# сбор информации
url = 'https://education.yandex.ru/universities/university/'
university_id = 2211 # id МГУ
university = requests.get(url + str(university_id))
#university.ok

soup = BeautifulSoup(university.text, 'html.parser')

uni = soup.find('h1', attrs={'class': ['university-title__title']})
print(uni.text)

table = soup.find('table', attrs={'class': ['year-stats__table']})
tr_list = table.findAll('tr')

data = []

flag_no_department = False # нет информации о кафедре
flag_general = False # указаны общие данные

for tr in tr_list:
    td_list = tr.findAll('td')
    
    if tr.attrs['class'] == ['year-stats-table-row', 'year-stats-table-row_bold', 'year-stats__data-row']:
        flag_no_department = td_list[1].text == 'Специальности, факультет для которых уточните в приемной комиссии'
        
        if not flag_no_department:
            level = get_level(td_list)
            specialty = get_specialty(td_list)
            
            flag_general = len(td_list) > 2
            
            if flag_general:
                subjects = get_subjects(td_list)
                budget_plan = get_budget_plan(td_list)
                budget = get_budget(td_list)
                budget_passing_score = get_budget_passing_score(td_list)
                paid_plan = get_paid_plan(td_list)
                paid = get_paid(td_list)
                paid_passing_score = get_paid_passing_score(td_list)
                cost = get_cost(td_list)
    
    elif tr.attrs['class'] == ['year-stats-table-row', 'year-stats__data-row']:
        if flag_no_department:
            level = get_level(td_list)
            specialty = get_specialty(td_list)
            
            department = None
        else:
            department = get_department(td_list)
         
        if not flag_general:
            subjects = get_subjects(td_list)
            budget_plan = get_budget_plan(td_list)
            budget = get_budget(td_list)
            budget_passing_score = get_budget_passing_score(td_list)
            paid_plan = get_paid_plan(td_list)
            paid = get_paid(td_list)
            paid_passing_score = get_paid_passing_score(td_list)
            cost = get_cost(td_list)
    
        specialty_info = {'level': level,
                          'specialty': specialty,
                          'department': department,
                          'subjects': subjects,
                          'budget_plan': budget_plan,
                          'budget': budget,
                          'budget_passing_score': budget_passing_score,
                          'paid_plan': paid_plan,
                          'paid': paid,
                          'paid_passing_score': paid_passing_score,
                          'cost': cost}
        
        data.append(specialty_info)
    
    else:
        #throw exception
        break

table = pd.DataFrame(data)
table = table[['specialty', 'department', 'level', 'subjects', 'budget_plan', 'budget', 'budget_passing_score', 'paid_plan', 'paid', 'paid_passing_score', 'cost']]
#table = table.drop(columns=['budget', 'paid'])
table = table.set_index('specialty')
table
#writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
#table.to_excel(writer,'Sheet1')
#writer.save()


Алтайский государственный педагогический университет


,department,level,subjects,budget_plan,budget,budget_passing_score,paid_plan,paid,paid_passing_score,cost
specialty,,,,,,,,,,
Прикладная математика,институт физико-математического образования,Бакалавриат,"[математика, русский язык, физика]",24,"{'По конкурсу': '23', 'Целевой прием': '0', 'П...",133,2,"{'По конкурсу': '2', 'Целевой прием': '0', 'По...",118,77680
Прикладная информатика,институт физико-математического образования,Бакалавриат,"[математика, русский язык, физика]",25,"{'По конкурсу': '21', 'Целевой прием': '3', 'П...",142,2,"{'По конкурсу': '1', 'Целевой прием': '0', 'По...",128,83000
Педагогическое образование,факультет физической культуры,Бакалавриат,"[обществознание, русский язык]",23,"{'По конкурсу': '17', 'Целевой прием': '5', 'П...",219,30,"{'По конкурсу': '19', 'Целевой прием': '0', 'П...",139,78440
Педагогическое образование,филологический факультет,Бакалавриат,"[литература, обществознание, русский язык]",25,"{'По конкурсу': '15', 'Целевой прием': '9', 'П...",206,15,"{'По конкурсу': '37', 'Целевой прием': '0', 'П...",111,78440
Педагогическое образование,филологический факультет,Бакалавриат,"[литература, обществознание, русский язык]",25,"{'По конкурсу': '14', 'Целевой прием': '8', 'П...",205,None,None,None,
Психолого-педагогическое образование,институт психологии и педагогики,Бакалавриат,"[биология, математика, русский язык]",22,"{'По конкурсу': '14', 'Целевой прием': '5', 'П...",159,1,"{'По конкурсу': '5', 'Целевой прием': '0', 'По...",118,78440
Специальное (дефектологическое) образование,институт психологии и педагогики,Бакалавриат,"[биология, математика, русский язык]",40,"{'По конкурсу': '26', 'Целевой прием': '10', '...",158,2,"{'По конкурсу': '11', 'Целевой прием': '0', 'П...",130,78440
Педагогическое образование (с двумя профилями подготовки),институт психологии и педагогики,Бакалавриат,"[математика, обществознание, русский язык]",23,"{'По конкурсу': '13', 'Целевой прием': '7', 'П...",181,1,"{'По конкурсу': '1', 'Целевой прием': '0', 'По...",133,78440
Педагогическое образование (с двумя профилями подготовки),институт психологии и педагогики,Бакалавриат,"[математика, обществознание, русский язык]",68,"{'По конкурсу': '38', 'Целевой прием': '21', '...",161,3,"{'По конкурсу': '16', 'Целевой прием': '0', 'П...",106,78440


In [26]:
# формирование запроса для получения списка университетов в регионе
#region = {438: {'name': 'Алтайский край', 'universities': []}}
#region = {439: {'name': 'Амурская область', 'universities': []}}
region_id = 467

url = 'https://education.yandex.ru/universities/graphql'

payload = {
    "query": "query SearchQuery($year: Int!, $subjectsStats: [SubjectScoreInput!]!, $possibleEducationTypes: [EDUCATION_TYPE!]!, $specialtyIds: [Int!], $regionsIds: [Int!], $requireMilitaryReserve: Boolean, $requireMilitaryContract: Boolean, $requireDormitory: Boolean) {\n  search(input: {year: $year, subjectsStats: $subjectsStats, possibleEducationTypes: $possibleEducationTypes, specialtyIds: $specialtyIds, regionsIds: $regionsIds, requireMilitaryReserve: $requireMilitaryReserve, requireMilitaryContract: $requireMilitaryContract, requireDormitory: $requireDormitory}) {\n    id\n    title\n    abbreviation\n    city\n    showCity\n    parent {\n      id\n      title\n      abbreviation\n      __typename\n    }\n    admissions {\n      id\n      srcYear\n      subjectIds\n      budgetMinPointsSum\n      paidMinPointsSum\n      tuition\n      creativeCompetitions\n      faculties {\n        id\n        title\n        abbreviation\n        __typename\n      }\n      specialtyIds\n      __typename\n    }\n    __typename\n  }\n}\n",
    "variables": {"year": "2017", "subjectsStats": [],
                  "specialtyIds": [1478,1479,1480,1481,1482,1483,1484,1485,1486,1487,
                                   1488,1489,1490,1491,1492,1493,1494,1495,1496,1497,
                                   1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,
                                   1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,
                                   1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,
                                   1528,1529,1530,1531,1532,1533,1534,1535,1536,1537,
                                   1538,1539,1540,1541,1542,1543,1544,1545,1546,1547,
                                   1548,1549,1550,1551,1552,1553,1554,1555,1556,1557,
                                   1558,1559,1560,1561,1562,1563,1564,1565,1566,1567,
                                   1568,1569,1570,1571,1572,1573,1574,1575,1576,1577,
                                   1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,
                                   1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,
                                   1598,1599,1600,1601,1602,1603,1604,1605,1606,1607,
                                   1608,1609,1610,1611,1612,1613,1614,1615,1616,1617,
                                   1618,1619,1620,1621,1622,1623,1624,1625,1633,1634,
                                   1635,1626,1627,1628,1629,1630,1631,1632,1636,1637,
                                   1638,1639,1640,1644,1645,1646,1641,1642,1643,1647,
                                   1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,
                                   1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,
                                   1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,
                                   1678,1679,1680,1681,1682,1683,1684,1685,1686,1687,
                                   1688,1689,1690,1691,1692,1693,1694,1698,1699,1700,
                                   1701,1702,1703,1695,1696,1697,1704,1705,1706,1707,
                                   1708,1709,1710,1711,1712,1713,1714,1716,1717,1718,
                                   1715,1719,1720,1721,1722,1723,1724,1725,1726,1727,
                                   1728,1729,1730,1731,1732,1733,1734,1735,1736,1737,
                                   1738,1739,1740,1741,1742,1743,1744,1745,1746,1747,
                                   1748,1749,1750,1751,1752,1753,1754,1755,1756,1757,
                                   1758,1759,1760,1761,1762,1763,1764,1765,1767,1769,
                                   1770,1771,1766,1768,1772],
                  "possibleEducationTypes": ["APPLIED_BACHELOR", "ACADEMIC_BACHELOR", "SPECIALTY"],
                  "regionsIds": [],
                  "requireMilitaryReserve": False, "requireMilitaryContract": False, "requireDormitory": False},
    "operationName": "SearchQuery"}

headers = {
    'Content-Type': 'application/json',
}

#for region_id in regions.keys():
payload['variables']['regionsIds'] = [region_id]
response = requests.post(url, headers=headers,
                         data=json.dumps(payload))
#regions[region_id]['universities'].append(list(map(lambda x: x['id'], response.json()['data']['search'])))
universitys_id = list(map(lambda x: x['id'], response.json()['data']['search']))

print(universitys_id)

[2485, 2415, 2476, 2410, 2489, 2280, 2380, 2486, 2389, 2575, 2391, 2482, 2401, 2398, 2399, 2381, 2416, 2364, 2278, 2492, 2414, 2265, 2384, 2393, 2394, 2395, 2377, 2371, 2480, 2383, 2379, 2479, 2658, 2457, 2390, 2274, 2493, 2400, 2388, 2387, 2281, 2611, 2397, 2385, 2487, 2273, 2382, 2411, 2477, 2376, 2266, 2378, 2488, 2392, 2494, 2490, 2478, 2386, 2396, 2418, 2403, 2275]


In [40]:
url_yandex = 'https://education.yandex.ru/universities/university/'

url_vuzoteka = 'http://vuzoteka.ru/%D0%B2%D1%83%D0%B7%D1%8B/vuzi-form'
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

universities_id = [2485, 2415, 2476, 2410, 2489, 2280, 2380, 2486, 2389, 2575, 2391, 2482, 2401, 2398, 2399, 2381, 2416, 2364, 2278, 2492, 2414, 2265, 2384, 2393, 2394, 2395, 2377, 2371, 2480, 2383, 2379, 2479, 2658, 2457, 2390, 2274, 2493, 2400, 2388, 2387, 2281, 2611, 2397, 2385, 2487, 2273, 2382, 2411, 2477, 2376, 2266, 2378, 2488, 2392, 2494, 2490, 2478, 2386, 2396, 2418, 2403, 2275]

for university_id in universities_id:
    university_page = requests.get(url_yandex + str(university_id))
    
    soup = BeautifulSoup(university_page.text, 'html.parser')
    
    university_name = soup.find('h1', attrs={'class': ['university-title__title']}).text
    count = 3
    response = requests.post(url_vuzoteka, headers=headers, data={'q': university_name[:count]})
    size = response.json()['foundCount']
    stop_counter = 0
    while size == -1 or (size > 1 and stop_counter < 40):
        count += 1
        response = requests.post(url_vuzoteka, headers=headers, data={'q': university_name[:count]})
        size_new = response.json()['foundCount']
        if size_new != -1 and size_new == size:
            stop_counter += 1
        size = size_new
    
    if size == 1:
        print(university_name, '\n', response.json()['data'][0]['place'])
    else:
        print(university_name, '\n', None)
        
    sleep(np.random.random() % 3)

Российский государственный социальный университет 
 86
Национальный исследовательский университет «Московский государственный институт электронной техники» 
 None
Российская академия народного хозяйства и государственной службы при Президенте РФ 
 None
Национальный исследовательский Московский государственный строительный университет 
 36
Российский государственный университет туризма и сервиса 
 146
Государственный университет по землеустройству 
 156
Московский архитектурный институт (государственная академия) 
 235
Российский государственный университет им. А. Н. Косыгина (Технологии. Дизайн. Искусство) 
 2006
Московский государственный областной университет 
 67
Технологический университет 
 None
Московский государственный технический университет им. Н. Э. Баумана 
 5
Российский государственный гуманитарный университет 
 65
Московский технологический университет 
 83
Московский педагогический государственный университет 
 52
Московский политехнический университет 
 51
Московский го

In [22]:
a = {'a': 2, 'b': 4, 'c': 3}
list(a.keys())

['a', 'b', 'c']

In [5]:
a = ['a', 'b', 'c']
b = 'b'
print(b in a)

True
